# Runner for Experiment 1


## Experiment

In [6]:
from qiskit import assemble
from qiskit.transpiler import CouplingMap
from qiskit.transpiler.passes import CSPLayout, CspRbsLayout, DenseLayout, \
                                     NoiseAdaptiveLayout, SabreLayout
from qiskit.transpiler.passes.layout import LCSLayout
from qiskit.providers.aer import QasmSimulator
from qiskit.providers.aer.noise import NoiseModel

from exp.custom_passmanager import custom_pass_manager

In [11]:
def exp1_evaluate(circuit, layout_method, backend, ideal=True, shots=8192, seed=None, pass_reps=100):
 
    coupling_map = CouplingMap(backend.configuration().coupling_map)
    backend_properties = backend.properties()

    if layout_method == 'csplayout':
        layout = CSPLayout(coupling_map, seed=seed, call_limit=None, time_limit=None)
    elif layout_method == 'csprbslayout':
        layout = CspRbsLayout(coupling_map, seed=seed, call_limit=None, time_limit=None, \
                              solution_limit=5, backend_prop=backend_properties)
    elif layout_method == 'lcslayout':
        layout = LCSLayout(coupling_map, seed=seed, call_limit=None, time_limit=None)
    elif layout_method == 'dense':
        layout = DenseLayout(coupling_map, None if ideal else backend_properties)
    elif layout_method == 'noise_adaptive':
        layout = NoiseAdaptiveLayout(backend_properties)
    elif layout_method == 'sabre':
        layout = SabreLayout(coupling_map, max_iterations=1, seed=seed)
    else:
        raise Exception('layout_method unknown %s' % layout_method)

    if ideal:
        simulator = QasmSimulator(configuration=backend.configuration(),
                                  method='stabilizer')
    else:
        noise_model = NoiseModel.from_backend(backend, thermal_relaxation=False)
        simulator = QasmSimulator(configuration=backend.configuration(),
                                  noise_model=noise_model,
                                  method='stabilizer')

    passmanager = custom_pass_manager(backend, layout, layout_reps=pass_reps)

    times = {}
    count_ops_after_map = {}

    def callback(**kwargs):
        print(kwargs["property_set"]["layout"])
        times[kwargs['pass_'].name()] = times.get(kwargs['pass_'].name(), 0) + kwargs['time']
        if kwargs['pass_'].name() == 'StochasticSwap':
            count_ops_after_map.update(kwargs['dag'].count_ops())
    
    transpiled = passmanager.run(circuit, callback=callback)

    needed_swaps = 0 if passmanager.property_set['is_swap_mapped'] else count_ops_after_map['swap']

    qobj = assemble(transpiled, backend, shots=shots, seed_simulator=0)
    result = simulator.run(qobj).result()
    counts = result.get_counts()
    time = times[layout.name()] / pass_reps

    return counts, time, needed_swaps

## Runner

In [12]:
import pandas as pd
from qiskit.test.mock.backends import FakeManhattan
from qiskit.providers.aer import QasmSimulator

from exp.circuit_generators import graphstate_manhatten_ring_hm
from exp.write_to_df import write_to_dataframe

In [13]:
def exp1_run(layout_method, exp_path, num_samples=100, num_qubits=12, num_shots=8192):
    backend = FakeManhattan()
    circuit = graphstate_manhatten_ring_hm(num_qubits)

    df = pd.DataFrame(columns=["seed", "state", "ideal_count", "noise_count", \
                           "ideal_time", "noise_time", "ideal_swaps_needed", "noise_swaps_needed"])
    for num_index in range(num_samples):
        seed = num_index
        ideal_counts, ideal_time, swaps_needed_ideal = exp1_evaluate(
            circuit, layout_method, backend=backend, ideal=True, shots=num_shots, seed=seed)
        noise_counts, noise_time, swaps_needed_noise = exp1_evaluate(
            circuit, layout_method, backend=backend, ideal=False, shots=num_shots, seed=seed)
        total_result = {'seed': seed,
                        'ideal_counts': ideal_counts,
                        'noise_counts': noise_counts,
                        'ideal_time': ideal_time,
                        'noise_time': noise_time,
                        'ideal_swaps_needed': swaps_needed_ideal,
                        'noise_swaps_needed': swaps_needed_noise}

        df_loc = write_to_dataframe(total_result)
        df = df.append(df_loc)

    df.to_csv(exp_path + f'exp1_{layout_method}.csv')

## Execution

In [14]:
layout_methods = [
    #'csplayout',
    'csprbslayout',
    #'dense',
    #'noise_adaptive',
    #'sabre',
    #'lcslayout'
]
exp_path = "./data/"
num_samples = 100
for method in layout_methods:
    exp1_run(method, exp_path, num_samples=num_samples)

 'ancilla'), 13),
26: Qubit(QuantumRegister(53, 'ancilla'), 14),
27: Qubit(QuantumRegister(53, 'ancilla'), 15),
28: Qubit(QuantumRegister(53, 'ancilla'), 16),
29: Qubit(QuantumRegister(53, 'ancilla'), 17),
30: Qubit(QuantumRegister(53, 'ancilla'), 18),
31: Qubit(QuantumRegister(53, 'ancilla'), 19),
32: Qubit(QuantumRegister(53, 'ancilla'), 20),
33: Qubit(QuantumRegister(53, 'ancilla'), 21),
34: Qubit(QuantumRegister(53, 'ancilla'), 22),
35: Qubit(QuantumRegister(53, 'ancilla'), 23),
36: Qubit(QuantumRegister(53, 'ancilla'), 24),
37: Qubit(QuantumRegister(53, 'ancilla'), 25),
38: Qubit(QuantumRegister(53, 'ancilla'), 26),
39: Qubit(QuantumRegister(53, 'ancilla'), 27),
40: Qubit(QuantumRegister(53, 'ancilla'), 28),
41: Qubit(QuantumRegister(53, 'ancilla'), 29),
42: Qubit(QuantumRegister(53, 'ancilla'), 30),
43: Qubit(QuantumRegister(53, 'ancilla'), 31),
44: Qubit(QuantumRegister(53, 'ancilla'), 32),
45: Qubit(QuantumRegister(53, 'ancilla'), 33),
46: Qubit(QuantumRegister(53, 'ancilla'), 

KeyError: "The item Qubit(QuantumRegister(12, 'q'), 0) does not exist in the Layout"